## Dependências

In [2]:
!pip install psycopg2-binary ipython-sql pandas
import pandas as pd
import psycopg2


# Configurações

In [20]:
from sqlalchemy import create_engine

db_config = {
    'user': 'postgres',
    'password': 'senha_supersegura_123',
    'host': 'localhost',
    'port': '5432'
}

engine = create_engine(
    f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_name}"
)


# Parte 3

## Tarefa 11

In [21]:
# 11a


from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT #necessário para a criação do DB que não seja feito em transações


conn = psycopg2.connect(dbname='postgres', **db_config) #conexão com postgres
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

db_name = "trabalho_bd3"
try:
    cur.execute(f"CREATE DATABASE {db_name};") #criando o db
    print(f"Banco '{db_name}' criado com sucesso!")
except psycopg2.errors.DuplicateDatabase:
    print(f"O banco '{db_name}' já existe.")
finally:
    cur.close()
    conn.close()

conn = psycopg2.connect(dbname=db_name, **db_config) #conectando com o db
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

#execução do script

try:
    # Lê o arquivo de forma segura
    with open('movie.sql', 'r') as f:
        sql_script = f.read()
    
    cur.execute(sql_script)
    print("Tabela 'movie' recriada e dados inseridos com sucesso!")
except psycopg2.errors.UndefinedTable:
    # Esse erro acontece se o script tiver "DROP TABLE movie" mas a tabela não existir
    print("Erro: O script tentou apagar uma tabela que não existe.")
    print("Dica: Altere a primeira linha do movie.sql para 'DROP TABLE IF EXISTS movie;'")
except Exception as e:
    print(f"Ocorreu um erro ao rodar o script: {e}")

O banco 'trabalho_bd3' já existe.
Tabela 'movie' recriada e dados inseridos com sucesso!


In [ ]:
# 11b
from sqlalchemy import create_engine
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
try:
    old_isolation = conn.isolation_level
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) # Vacuum não roda em transação
    
    cur = conn.cursor()
    cur.execute("VACUUM ANALYZE movie;")
    print("Estatísticas do banco atualizadas com sucesso!")
    
except Exception as e:
    print(f"Aviso: {e}")
    conn.rollback()

sql_querry = """
SELECT
    t.relname AS tabela,
    i.relname AS indice,
    pg_size_pretty(pg_relation_size(i.oid)) AS tamanho,
    meta.level AS altura_arvore,     -- Nível da raiz (0 = folha, >0 = ramos)
    meta.root AS bloco_raiz,         -- Qual bloco é a raiz
    c.relpages AS total_blocos,      -- Total de páginas (blocos) do índice
    c.reltuples AS total_linhas      -- Estimativa de linhas indexadas
FROM pg_class t
JOIN pg_index x ON t.oid = x.indrelid
JOIN pg_class c ON c.oid = x.indexrelid
JOIN pg_class i ON i.oid = x.indexrelid
-- A função bt_metap lê a 'meta página' do índice B-Tree para pegar a altura
CROSS JOIN LATERAL bt_metap(i.relname) as meta
WHERE t.relname = 'movie';
"""

try: 
    df_resultado = pd.read_sql(sql_querry, engine)

    print("Relatório de Metadados do Índice")
    display(df_resultado)

    nome_arquivo = "relatorio_tarefa_11.txt"
    
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        
        f.write("BANCOS DE DADOS 1 - TP3 - TAREFA 11\n")
        f.write("="*50 + "\n\n")
        f.write("RELATÓRIO DE METADADOS DOS ÍNDICES:\n\n")
        
        # O método .to_string() converte a tabela do Pandas para texto puro
        # index=False remove a numeração das linhas (0, 1, 2...)
        tabela_texto = df_resultado.to_string(index=False, justify='left')
        
        f.write(tabela_texto)
        f.write("\n\n" + "="*50)
        
    print(f"\nSucesso! O arquivo '{nome_arquivo}' foi salvo na pasta do notebook.")

except Exception as e:
    print(f"Erro na consulta: {e}")

finally:
    cur.close()
    conn.close

Estatísticas do banco atualizadas com sucesso!
Relatório de Metadados do Índice


,tabela,indice,tamanho,altura_arvore,bloco_raiz,total_blocos,total_linhas
0,movie,movie_key,56 kB,1,3,7,1844.0
1,movie,movie_title,96 kB,1,3,12,1844.0
2,movie,movie_votes,88 kB,1,3,11,1844.0



Sucesso! O arquivo 'relatorio_tarefa_11.txt' foi salvo na pasta do notebook.


## Tarefa 12

In [23]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_teorica = """
RESPOSTA DA QUESTÃO 12.c:

O índice sobre o atributo VOTES não é utilizado em todas as consultas porque o otimizador
do PostgreSQL (Query Planner) avalia o custo total da operação.

1. Para consultas seletivas (poucas tuplas, <10), o custo de navegar na estrutura
   de árvore do índice (Index Scan) é baixo, e vale a pena para evitar ler a tabela toda.

2. Para consultas abrangentes (>80% das tuplas), o uso do índice geraria muitas operações
   de I/O aleatório (pular do índice para a tabela milhares de vezes). Nesse caso,
   é muito mais barato fazer um 'Sequential Scan' (ler a tabela inteira sequencialmente),
   aproveitando o cache do SO e a leitura em blocos contíguos.
"""

# Definição das Queries
queries = [
    {
        "titulo": "12.a) Consulta Pequena (<10 tuplas)",
        "sql": "SELECT title, votes FROM movie WHERE votes > 45000;" 
    },
    {
        "titulo": "12.b) Consulta Grande (>80% das tuplas)",
        "sql": "SELECT title, votes FROM movie WHERE votes > 10;"
    }
]

# execução e gravação do arquivo

nome_arquivo = "relatorio_tarefa_12.txt"

try:
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        
        f.write("BANCOS DE DADOS 1 - TRABALHO 3 - TAREFA 12\n")
        f.write("="*60 + "\n\n")

        # Loop para executar as duas consultas (a e b)
        for q in queries:
            print(f"Processando: {q['titulo']}...")
            
            # Escreve o título no arquivo
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY SQL: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa o EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            display(df)
            
            # Converte o DataFrame para string limpa (sem index, sem cabeçalho de coluna)
            # header=False remove o título 'QUERY PLAN' que é redundante
            plano_texto = df.to_string(index=False, header=False, justify='left')
            
            # Salva no arquivo
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # --- AQUI ENTRA A SUA RESPOSTA TEÓRICA ---
        f.write("ANÁLISE TEÓRICA (ITEM 12.c):\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_teorica)
        f.write("\n" + "="*60)

    print(f"\nSucesso! O arquivo '{nome_arquivo}' foi gerado com os planos e sua resposta.")

except Exception as e:
    print(f"Erro: {e}")

Processando: 12.a) Consulta Pequena (<10 tuplas)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes > 45000)
2,Planning Time: 0.177 ms
3,Execution Time: 0.018 ms


Processando: 12.b) Consulta Grande (>80% das tuplas)...


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..38.05 rows=1844...
1,Filter: (votes > 10)
2,Planning Time: 0.026 ms
3,Execution Time: 0.170 ms



Sucesso! O arquivo 'relatorio_tarefa_12.txt' foi gerado com os planos e sua resposta.


## Tarefa 13

In [27]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_tarefa_13 = """
RESPOSTA DA QUESTÃO 13 (Análise Comparativa):

a) Diferença entre os planos:
   SIM, houve uma diferença.
   - A Consulta A (MAX) utilizou um 'InitPlan' com 'Index Only Scan'. O banco calculou
     o valor máximo uma única vez (acessando o índice de forma invertida) e usou esse
     valor para filtrar a tabela.
   - A Consulta B (ALL) falhou em otimizar a lógica. O plano mostra um 'Seq Scan'
     (varredura sequencial) combinado com um 'SubPlan' que foi executado repetidamente
     (loops=1844), gerando um produto cartesiano lógico.

b) Qual das duas é mais eficiente?
   A Consulta A (MAX) é muito mais eficiente.
   - Custo estimado: ~35.37 (MAX) vs ~43620.99 (ALL).
   - Tempo real: 0.063ms (MAX) vs 0.632ms (ALL).

c) Explicação:
   Embora as consultas pareçam equivalentes, o operador 'ALL' possui uma semântica
   complexa de tratamento de valores NULL (se houver um NULL na subconsulta, o resultado
   pode ser indeterminado). O otimizador do PostgreSQL, neste cenário, não conseguiu
   garantir que a transformação de 'ALL' para 'MAX' seria segura ou vantajosa, apesar de
   não haver NULLs no DB (apenas ). Ele deicidiu optar por um plano genérico de verificação
   linha-a-linha (nested loop/materialize), o que elevou drasticamente o custo computacional.
"""

queries = [
    {
        "titulo" : "Consulta A - Usando MAX",
        "sql" : "SELECT title FROM movie WHERE votes >= (SELECT MAX(votes) FROM movie);"
    }, {
        "titulo" : "Consulta B - Usando ALL",
        "sql" : "SELECT title FROM movie WHERE votes >= ALL (SELECT votes FROM movie);"
    }
]

nome_arquivo = "relatorio_tarefa_13.txt"
try: 
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCO DE DADOS 1 - TRABALHO 3 - TAREFA 13\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Analisando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            # Mostra na tela para conferência rápida
            display(df)
            
            # Grava no arquivo
            plano_texto = df.to_string(index=False, header=False, justify='left')
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # Adiciona a resposta teórica no final
        f.write("ANÁLISE E RESPOSTAS:\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_tarefa_13)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro: {e}")

finally:
    conn.close()

Analisando: Consulta A - Usando MAX...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes >= $1)
2,InitPlan 2 (returns $1)
3,-> Result (cost=0.32..0.33 rows=1 width=...
4,InitPlan 1 (returns $0)
5,-> Limit (cost=0.28..0.32 rows=1...
6,-> Index Only Scan Backward...
7,Index Cond: (votes IS ...
8,Heap Fetches: 0
9,Planning Time: 0.039 ms


Analisando: Consulta B - Usando ALL...


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..43620.99 rows=9...
1,Filter: (SubPlan 1)
2,Rows Removed by Filter: 1843
3,SubPlan 1
4,-> Materialize (cost=0.00..42.66 rows=18...
5,-> Seq Scan on movie movie_1 (cost...
6,Planning Time: 0.057 ms
7,Execution Time: 0.856 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_13.txt'.


## Tarefa 14

In [37]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_tarefa_13 = """
RESPOSTA DA TAREFA 14 (Junção vs Seleção/Subconsulta):

a) Resultado do EXPLAIN ANALYZE:
   (Os planos detalhados estão apresentados acima).

b) Existe diferença entre os planos?
   SIM. Embora a lógica seja equivalente, a estrutura dos planos diferiu:
   
   1. Consulta A (Subconsulta): Utilizou um "InitPlan". O banco calculou os votos
      de 'Star Wars' ($0) isoladamente e antes da execução principal. Em seguida,
      usou esse valor escalar para realizar um 'Index Scan' simples na tabela movie.
      
   2. Consulta B (Junção): Utilizou um "Nested Loop" (Loop Aninhado). O banco
      primeiro localizou a tupla de 'Star Wars' (m2) e usou os dados dessa tupla
      para dirigir a busca na tabela m1.

c) Qual das duas é mais eficiente?
   Neste teste específico, a Consulta A (Subconsulta) foi ligeiramente mais eficiente.
   
   - Custo Total Estimado: 43.34 (A) vs 49.49 (B).
   - Tempo de Execução: 0.025ms (A) vs 0.034ms (B).
   
   Explicação: A estratégia de "InitPlan" (A) é extremamente otimizada para casos
   onde o filtro é um valor escalar constante. O "Nested Loop" (B), embora eficiente,
   possui um overhead ligeiramente maior para gerenciar a estrutura de junção,
   mesmo quando o loop externo retorna apenas uma linha. Contudo, para fins práticos
   em sistemas reais, ambas são consideradas de alta performance (sub-milissegundo).
"""

queries = [
    {
        "titulo": "14.a) Usando Subconsulta (Subquery)",
        "sql": "SELECT title FROM movie WHERE votes > (SELECT votes FROM movie WHERE title = 'Star Wars')"
    },
    {
        "titulo": "14.b) Usando Junção (Self-Join)",
        "sql": "SELECT m1.title FROM movie m1, movie m2 WHERE m1.votes > m2.votes AND m2.title = 'Star Wars'"
    }
]

nome_arquivo = "relatorio_tarefa_14.txt"
try: 
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCO DE DADOS 1 - TRABALHO 3 - TAREFA 14\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Analisando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            # Mostra na tela para conferência rápida
            display(df)
            
            # Grava no arquivo
            plano_texto = df.to_string(index=False, header=False, justify='left')
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # Adiciona a resposta teórica no final
        f.write("ANÁLISE E RESPOSTAS:\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_tarefa_13)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro: {e}")

finally:
    conn.close()

Analisando: 14.a) Usando Subconsulta (Subquery)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=8...
1,Index Cond: (votes > $0)
2,InitPlan 1 (returns $0)
3,-> Index Scan using movie_title on movie ...
4,Index Cond: ((title)::text = 'Star W...
5,Planning Time: 0.036 ms
6,Execution Time: 0.020 ms


Analisando: 14.b) Usando Junção (Self-Join)...


,QUERY PLAN
0,Nested Loop (cost=0.56..49.49 rows=615 width=...
1,-> Index Scan using movie_title on movie m2...
2,Index Cond: ((title)::text = 'Star War...
3,-> Index Scan using movie_votes on movie m1...
4,Index Cond: (votes > m2.votes)
5,Planning Time: 0.035 ms
6,Execution Time: 0.017 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_14.txt'.


## Tarefa 15

In [35]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)

resposta_teorica = """
RESPOSTA DA TAREFA 15 (Casamento de Strings e Índices):
RESPOSTA DA TAREFA 15 (Casamento de Strings e Índices):

a) Resultados do EXPLAIN:
   Todas as três consultas resultaram em 'Seq Scan' (Varredura Sequencial). O índice B-Tree
   sobre 'TITLE' não foi utilizado em nenhum dos casos.

b) Qual das três apresenta o menor custo?
   As consultas 15.1 (LIKE 'I%') e 15.3 (LIKE '%A') empataram com o menor custo estimado (38.05).
   A consulta 15.2 (substr) teve o MAIOR custo (42.66).
   
   Isso ocorre porque a função 'substr' exige processamento de CPU adicional para calcular
   a substring de cada linha antes de comparar, elevando o custo em relação à comparação
   simples de texto das outras duas. O tempo de execução real confirmou isso (0.168ms para substr
   vs 0.094ms para LIKE 'I%').

c) O índice sobre TITLE foi usado para todas elas? Justifique.
   NÃO. O índice não foi usado em nenhuma.

   1. LIKE 'I%': Teoricamente, índices B-Tree suportam buscas por prefixo. Porém, no PostgreSQL,
      se o Locale do banco não for 'C' (ex: pt_BR ou en_US), o índice padrão não suporta o operador LIKE
      devido a regras complexas de colação (ordenação). O otimizador, portanto, reverteu para Seq Scan.
      (Nota: Para funcionar, o índice precisaria ter sido criado com 'varchar_pattern_ops').

   2. substr(title, 1, 1): Índices B-Tree não suportam funções. Ao aplicar 'substr',
      o banco perde a referência da chave indexada e precisa calcular a função linha a linha.

   3. LIKE '%A': Índices B-Tree são ordenados pelo início da string. Como o curinga '%' está
      no início, a árvore não serve para a busca, forçando a leitura completa da tabela.
"""

queries = [
    {
        "titulo": "15.1) LIKE 'I%' (Tudo)",
        "sql": "SELECT title FROM movie WHERE title LIKE 'I%';"
    },
    {
        "titulo": "15.2) Substring (Função)",
        "sql": "SELECT title FROM movie WHERE substr(title, 1, 1) = 'I';"
    },
    {
        "titulo": "15.3) LIKE '%A' (Sufixo/Termina com A)",
        "sql": "SELECT title FROM movie WHERE title LIKE '%A';"
    }
]

# --- EXECUÇÃO E GERAÇÃO DO RELATÓRIO ---
nome_arquivo = "relatorio_tarefa_15.txt"

try:
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCOS DE DADOS 1 - TRABALHO 3 - TAREFA 15\n")
        f.write("CASAMENTO DE STRINGS E USO DE ÍNDICES\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Processando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            
            try:
                df = pd.read_sql(sql_explain, conn)
                display(df)
                plano_texto = df.to_string(index=False, header=False, justify='left')
                f.write(plano_texto)
            except Exception as e:
                f.write(f"Erro: {e}")
                
            f.write("\n\n" + "="*60 + "\n\n")
        
        f.write("ANÁLISE E RESPOSTAS (ITEM 15):\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_teorica)
        f.write("\n" + "="*60)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro geral: {e}")

finally:
    conn.close()

Processando: 15.1) LIKE 'I%' (Tudo)...


/tmp/ipykernel_16913/492076231.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_explain, conn)


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..38.05 rows=18 w...
1,Filter: ((title)::text ~~ 'I%'::text)
2,Rows Removed by Filter: 1819
3,Planning Time: 0.228 ms
4,Execution Time: 0.091 ms


Processando: 15.2) Substring (Função)...


/tmp/ipykernel_16913/492076231.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_explain, conn)


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..42.66 rows=9 wi...
1,"Filter: (substr((title)::text, 1, 1) = 'I'::..."
2,Rows Removed by Filter: 1819
3,Planning Time: 0.040 ms
4,Execution Time: 0.150 ms


Processando: 15.3) LIKE '%A' (Sufixo/Termina com A)...


/tmp/ipykernel_16913/492076231.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_explain, conn)


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..38.05 rows=18 w...
1,Filter: ((title)::text ~~ '%A'::text)
2,Rows Removed by Filter: 1814
3,Planning Time: 0.054 ms
4,Execution Time: 0.166 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_15.txt'.


## Tarefa 16

In [41]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_tarefa_13 = """

a) Hipótese Uniforme:
O PostgreSQL não utilizou uma hipótese uniforme global. As estimativas apresentadas 
no EXPLAIN (rows=328 e rows=8) ficaram muito próximas dos valores reais (326 e 4), o que 
mostra que o otimizador consultou estatísticas reais armazenadas em pg_stats, como histogramas 
e valores mais comuns. No entanto, dentro de cada intervalo (bucket) do histograma, o 
PostgreSQL assume distribuição uniforme, e essa combinação de histogramas reais mais uniformidade interna 
explica por que as estimativas ficaram tão próximas dos valores observados.

b) Menor Seletividade:
A consulta "votes < 1000" possui a menor seletividade.
  Alta seletividade significa filtrar muito e retornar poucos dados (como "votes > 40000").
  Baixa seletividade significa filtrar pouco e retornar muitos dados (como "votes < 1000").

"""

queries = [
    {
        "titulo": "16.a) Votes < 1000 (muitos filmes)",
        "sql": "SELECT title FROM movie WHERE votes < 1000;"
    },
    {
        "titulo": "16.b) Votes > 4000 (poucos filmes)",
        "sql": "SELECT title FROM movie WHERE votes > 40000;"
    }
]

nome_arquivo = "relatorio_tarefa_16.txt"
try: 
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCO DE DADOS 1 - TRABALHO 3 - TAREFA 14\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Analisando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            # Mostra na tela para conferência rápida
            display(df)
            
            # Grava no arquivo
            plano_texto = df.to_string(index=False, header=False, justify='left')
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # Adiciona a resposta teórica no final
        f.write("ANÁLISE E RESPOSTAS:\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_tarefa_13)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro: {e}")

finally:
    conn.close()

Analisando: 16.a) Votes < 1000 (muitos filmes)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes < 1000)
2,Planning Time: 0.015 ms
3,Execution Time: 0.036 ms


Analisando: 16.b) Votes > 4000 (poucos filmes)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes > 40000)
2,Planning Time: 0.022 ms
3,Execution Time: 0.008 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_16.txt'.
